# Yelp Dataset Recommendation v3

### Librerías

In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

### Lectura datasets

In [4]:
yelp_business_text = pd.read_csv("yelp_business_text.csv")
yelp_business_text.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,features_to_embed
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"Categories: Bubble Tea, Coffee & Tea, Bakeries..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,"Categories: Burgers, Fast Food, Sandwiches, Ic..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,"Categories: Pubs, Italian, Bars, American (Tra..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,"Categories: Ice Cream & Frozen Yogurt, Fast Fo..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"Categories: Vietnamese, Food Trucks. Attribute..."


In [5]:
yelp_reviews = pd.read_csv("yelp_reviews.csv")
yelp_reviews.head()

,review_id,user_id,business_id,stars,text
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,Rated 3.0 stars out of 5. If you decide to eat...
1,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,Rated 3.0 stars out of 5. Family diner. Had th...
2,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,"Rated 5.0 stars out of 5. Wow! Yummy, differe..."
3,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,Rated 4.0 stars out of 5. Cute interior and ow...
4,JrIxlS1TzJ-iCu79ul40cQ,eUta8W_HdHMXPzLBBZhL1A,04UD14gamNjLY0IDYVhHJg,1.0,Rated 1.0 stars out of 5. I am a long term fre...


### Embeddings reseñas pooled

In [7]:
def review_embeddings_training(business_dataset, reviews_dataset, merge_id = "business_id", text_column = "text", return_individual_embeddings = False):
    from sentence_transformers import SentenceTransformer

    # 1. Unir datasets por ID
    print("Merging datasets...")
    reviews_df_gouped = reviews_dataset.groupby(merge_id)[text_column].apply(list).reset_index()
    df = pd.merge(business_dataset, reviews_df_gouped, on=merge_id, how="left")
    print("Merge complete")

    # 2. Obtención de los embeddings individuales
    model = SentenceTransformer('all-MiniLM-L6-v2')

    all_ind_reviews = [review for sublist in df.text for review in sublist]
    print("Total reviews to embed:", len(all_ind_reviews))

    print("Embed in process:")
    ind_reviews_emb = model.encode(all_ind_reviews,
                               show_progress_bar = True)
    print("Embedding completed")
    print("Embeddings shape:", ind_reviews_emb.shape)

    # 2. Pooling medio por restaurante
    business_pooled_emb = []
    current_emb_idx = 0

    print("Pooling embeddings...")
    for index, row in df.iterrows():
        num_rev_business = len(row[text_column])

        start_idx = current_emb_idx
        end_idx = current_emb_idx + num_rev_business

        emb_current_business = ind_reviews_emb[start_idx:end_idx]
        pooled_emb = np.mean(emb_current_business, axis = 0)

        business_pooled_emb.append(pooled_emb)

        current_emb_idx = end_idx
    print("Pooling completed")
    
    # Salida
    if return_individual_embeddings:
        return ind_reviews_emb, business_pooled_emb
    
    return business_pooled_emb

In [8]:
restaurant_pooled_emb = review_embeddings_training(yelp_business_text, yelp_reviews)


Merging datasets...
Merge complete
Total reviews to embed: 4637828
Embed in process:


Batches: 100%|██████████| 144933/144933 [51:20<00:00, 47.04it/s] 


Embedding completed
Embeddings shape: (4637828, 384)
Pooling embeddings...
Pooling completed


#### Guardado de embeddings

In [13]:
df_embeddings = pd.DataFrame(restaurant_pooled_emb)
df_embeddings.head()
filename = "restaurant_reviews_pooled_embeddings.csv"
df_embeddings.to_csv(filename, index = False)
print("Embeddings guardados en", filename)

Embeddings guardados en restaurant_reviews_pooled_embeddings.csv


#### Lectura de embeddings y test

In [14]:
restaurant_pooled_emb = pd.read_csv("restaurant_reviews_pooled_embeddings.csv")
restaurant_pooled_emb = restaurant_pooled_emb.to_numpy()

In [17]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

query = "Im looking for the best food trucks in America"
query_embedding = model.encode([query])

cosine_similarities = cosine_similarity(query_embedding, restaurant_pooled_emb)[0]
top_n = cosine_similarities.argsort()[-10:][::-1]

res = yelp_business_text.iloc[top_n][["name", "address", "city", "state", "stars", "features_to_embed"]]
res

,name,address,city,state,stars,features_to_embed
9626,George's Food truck,3700 Sansom St,Philadelphia,PA,4.5,"Categories: Food Trucks, Food Stands. Attribut..."
26560,Semo's Cart,North Jefferson St W 11th St,Wilmington,DE,4.5,Categories: Greek. Attributes: Good for groups...
27049,Sailin' On,100 AVE AND 108 ST,Edmonton,AB,4.5,"Categories: Vegan, Food Trucks. Attributes: De..."
26774,Denise's Soul Food Restaurant,3025 Chestnut St,Philadelphia,PA,4.0,"Categories: Food Stands, Soul Food, Caribbean,..."
3228,La Marqueza - Food Truck,Spring Garden St,Philadelphia,PA,4.5,"Categories: Mexican, Breakfast & Brunch, Food ..."
18340,Troy Food Truck,36TH St And Market St Corner,Philadelphia,PA,4.5,"Categories: Street Vendors, Kebab, Halal, Food..."
5845,Slider Station,NaN,Indianapolis,IN,3.5,"Categories: Fast Food, Food Stands. Attributes..."
9328,Steaming Goat,NaN,Nashville,TN,4.5,"Categories: Food Trucks, Sandwiches. Attribute..."
20329,Taqueria El Indio,1505 12th Ave S,Nashville,TN,5.0,"Categories: Food Trucks, Mexican, Imported Foo..."
15569,A Quality Cafe,Walnut St,Philadelphia,PA,4.5,"Categories: Food Trucks, Halal, Falafel, Medit..."
